In [ ]:
import numpy as np
import idx2numpy
import matplotlib.pyplot as plt
import pandas as pd
import joblib
import os
np.random.seed(7)

## Analysis

In [ ]:
orig_X = idx2numpy.convert_from_file('Data/MNIST/train-images.idx3-ubyte')
y = idx2numpy.convert_from_file('Data/MNIST/train-labels.idx1-ubyte')

In [ ]:
for digit in range(10):
    samples = orig_X[y == digit]
    random_samples = np.random.choice(range(0, len(samples)), 1) # MAKE IT 5 !TODO
    for i in random_samples:
        plt.imshow(samples[i])
        plt.title(f'digit {digit}')
        plt.show()

## Data splitting

In [ ]:
def split(X, y, folds=5, shuffle=True):
    assert X.shape[0] == y.shape[0]
    n = X.shape[0]
    if shuffle:
        p = np.random.permutation(n)
        X, y = X[p], y[p]
    num = n // folds
    left = n % folds
    X_folds = []
    y_folds = []
    prev = 0
    for i in range(folds):
        length = num + (left > 0)
        X_ = np.copy(X[prev: prev + length])
        y_ = np.copy(y[prev: prev + length])
        X_folds.append(X_)
        y_folds.append(y_)
        prev += length
        left -= 1
    return X_folds, y_folds

def form_train_val(X_folds, y_folds, val_fold):
    train_X_folds = [X_folds[i] for i in range(len(X_folds)) if i != val_fold]
    train_y_folds = [y_folds[i] for i in range(len(y_folds)) if i != val_fold]
    train_X = np.concatenate(train_X_folds, axis=0)
    train_y = np.concatenate(train_y_folds, axis=0)
    val_X = np.copy(X_folds[val_fold])
    val_y = np.copy(y_folds[val_fold])
    return train_X, train_y, val_X, val_y

In [ ]:
X = np.array([np.ravel(r) for r in orig_X])

In [ ]:
X_folds, y_folds = split(X, y)

## Model & Training

In [ ]:
epsilon = 1e-6

def BCE(y_actual, y_pred):
    return -np.mean(y_actual * np.log(y_pred) + (1 - y_actual) * np.log(1 - y_pred))
    
def accuracy(y_actual, y_pred):
    return sum([y_pred[i] == y_actual[i] for i in range(len(y_actual))]) / len(y_actual)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mode(l):
    d = {0: 0}
    mode_ = 0
    for x in l:
        d[x] = (d[x] + 1) if (x in d) else 1
        if d[x] > d[mode_]:
            mode_ = x
    return mode_

In [ ]:
class LogRegression_Base:
    """
    Logistic Regression Base Class with L2 Regularisation
    Utilises Binary Cross Entropy loss as loss criterion
    Uses Batch Gradient Descent for updates
    """

    def __init__(self, normalise=True, lr=0.01, reg_lambda=0.01):
        self.normalise = normalise
        self.lr = lr
        self.reg_lambda = reg_lambda

    def fit(self, _X, y, _X_val=None, y_val=None, epochs=1000, precision=1e-6, display=False):
        if self.normalise:
            self.mean = np.mean(_X, axis=0)
            self.std = np.std(_X, axis=0) + epsilon
            _X = (_X - self.mean) / self.std
        X = np.c_[_X, np.ones(_X.shape[0])]
        
        validation = _X_val is not None
        X_val = None
        if validation:
            if self.normalise:
                _X_val = (_X_val - self.mean) / self.std
            X_val = np.c_[_X_val, np.ones(_X_val.shape[0])]

        self.num_samples, self.dim = X.shape

        self.w = np.zeros(self.dim)
        self.train_losses = []
        self.train_accuracies = []
        if validation:
            self.validation_losses = []
            self.validation_accuracies = []
        iters = 0
        prev_step_size = np.inf

        while iters < epochs and prev_step_size > precision:
            self.train_losses.append(self.loss(y, self.prob(X)))
            self.train_accuracies.append(accuracy(y, self.predict(_X)))
            if validation:
                self.validation_losses.append(self.loss(y_val, self.prob(X_val)))
                self.validation_accuracies.append(accuracy(y_val, self.predict(_X_val)))

            if display:
                print(f'Epoch {iters + 1} ----')
                print(f'Training loss: {self.train_losses[-1]}')
                print(f'Training accuracy: {self.train_accuracies[-1]}')
                if validation:
                    print(f'Validation loss: {self.validation_losses[-1]}')
                    print(f'Validation accuracy: {self.validation_accuracies[-1]}')

            w_ = self.w - self.lr * self.grad(X, y)
            prev_step_size = np.sum((self.w - w_) ** 2)
            self.w = w_
            iters += 1
        
        self.epochs_run = iters

    def loss(self, y_actual, y_pred):
        return BCE(y_actual, y_pred) + self.reg_lambda * np.sum(self.w ** 2)

    def prob(self, X):
        return sigmoid(X @ self.w)    

    def grad(self, X, y_actual):
        y_pred = self.prob(X)
        gradient = ((y_pred - y_actual) @ X) / self.num_samples + 2 * self.reg_lambda * self.w
        return gradient

    def predict(self, _X_test):
        if self.normalise:
            _X_test = (_X_test - self.mean) / self.std
        X_test = np.c_[_X_test, np.ones(_X_test.shape[0])]
        y_pred = np.array([int((self.w.T @ x) >= 0) for x in X_test])
        return y_pred

In [ ]:
class LogRegression:
    """
    Logistic regression class capable of handling muli class classification
    Uses LogRegression_Base as base class for logistic regression
    """

    def __init__(self, method, num_classes):
        self.method = method
        assert (self.method in ['Single', 'OVO', 'OVR'])
        self.num_classes = num_classes
        if self.num_classes == 2:
            self.method = 'Single'
        self.models = {}

    def fit_single(self, _X, y, _X_val, y_val, epochs):
        self.models[0] = LogRegression_Base()
        self.models[0].fit(_X, y, _X_val, y_val, epochs)

    def fit_ovr(self, _X, y, _X_val, y_val, epochs, display):
        if display:
            print('* Fitting OVR models')
        for c in range(self.num_classes):
            y_c = np.array([int(u) for u in (y == c)])
            y_val_c = np.array([int(u) for u in (y_val == c)])
            self.models[c] = LogRegression_Base()
            self.models[c].fit(_X, y_c, _X_val, y_val_c, epochs)
            if display:
                print(f'> Fitted model {c}')
                print(f'> Acc. train: {self.models[c].train_accuracies[-1]}')
                if _X_val is not None:
                    print(f'> Acc. val: {self.models[c].validation_accuracies[-1]}')
                print()
        if display:
            print('=========')

    def fit_ovo(self, _X, y, _X_val, y_val, epochs, display):
        if display:
            print('* Fitting OVO models')
        for c1 in range(self.num_classes):
            for c2 in range(c1 + 1, self.num_classes):
                idx = (y == c1) | (y == c2)
                idx_val = (y_val == c1) | (y_val == c2)
                X_c1_c2 = _X[idx]
                X_val_c1_c2 = _X_val[idx_val]
                y_c1_c2 = np.array([int(u == c1) for u in y[idx]])
                y_val_c1_c2 = np.array([int(u == c1) for u in y_val[idx_val]])
                self.models[(c1, c2)] = LogRegression_Base()
                self.models[(c1, c2)].fit(X_c1_c2, y_c1_c2, X_val_c1_c2, y_val_c1_c2, epochs)
                if display:
                    print(f'> Fitted model {c1}, {c2}')
                    print(f'> Acc. train: {self.models[(c1, c2)].train_accuracies[-1]}')
                    if _X_val is not None:
                        print(f'> Acc. val: {self.models[(c1, c2)].validation_accuracies[-1]}')
                    print()
        if display:
            print('=========')
    
    def predict_ovo(self, X):
        predictions = {}
        for c1 in range(self.num_classes):
            for c2 in range(c1 + 1, self.num_classes):
                p = self.models[(c1, c2)].predict(X)
                predictions[(c1, c2)] = [c1 if u else c2 for u in p]
        y_ = np.array(list(predictions.values()))
        y = np.array([mode(y_[:, i]) for i in range(X.shape[0])])
        return y
    
    def predict_ovr(self, X):
        prob = []
        for c in range(self.num_classes):
            prob.append(self.models[c].prob(X))
        prob_ = np.array(prob)
        y = np.array([np.argmax(prob_[:, i]) for i in range(X.shape[0])])
        return y
        
    def fit(self, _X, y, _X_val=None, y_val=None, epochs=500, display=True):
        if self.method == 'Single':
            self.fit_single(_X, y, _X_val, y_val, epochs)
        if self.method == 'OVO':
            self.fit_ovo(_X, y, _X_val, y_val, epochs, display)
        if self.method == 'OVR':
            self.fit_ovr(_X, y, _X_val, y_val, epochs, display)

    def predict(self, X):
        if self.method == 'Single':
            return self.models[0].predict(X)
        if self.method == 'OVO':
            return self.predict_ovo(X)
        if self.method == 'OVR':
            return self.predict_ovr(X)


In [ ]:
ovo_df = {'Validation fold': [], 'Train Acc.': [], 'Validation Acc.': []}
ovo_models_path = 'Weights/3/ovo-models.sav'
ovo_models_exists = os.path.exists(ovo_models_path)
ovo_models = joblib.load(ovo_models_path) if ovo_models_exists else []
for val_fold in range(len(X_folds)):
    t_X, t_y, v_X, v_y = form_train_val(X_folds, y_folds, val_fold)
    lr = None
    if not ovo_models_exists:
        lr = LogRegression(method='OVO', num_classes=10)
        lr.fit(t_X, t_y, v_X, v_y)
        ovo_models.append(lr)
    else:
        lr = ovo_models[val_fold]

    ovo_df['Validation fold'].append(val_fold)
    ovo_df['Train Acc.'].append(accuracy(t_y, lr.predict(t_X)))
    ovo_df['Validation Acc.'].append(accuracy(v_y, lr.predict(v_X)))
    
ovo_df = pd.Dataframe(ovo_df)
if not ovo_models_exists:
    joblib.dump(ovo_models, ovo_models_path)

In [ ]:
ovr_df = {'Validation fold': [], 'Train Acc.': [], 'Validation Acc.': []}
ovr_models_path = 'Weights/3/ovr-models.sav'
ovr_models_exists = os.path.exists(ovr_models_path)
ovr_models = joblib.load(ovr_models_path) if ovr_models_exists else []
for val_fold in range(len(X_folds)):
    t_X, t_y, v_X, v_y = form_train_val(X_folds, y_folds, val_fold)
    lr = None
    if not ovr_models_exists:
        lr = LogRegression(method='OVR', num_classes=10)
        lr.fit(t_X, t_y, v_X, v_y)
        ovr_models.append(lr)
    else:
        lr = ovr_models[val_fold]

    ovr_df['Validation fold'].append(val_fold)
    ovr_df['Train Acc.'].append(accuracy(t_y, lr.predict(t_X)))
    ovr_df['Validation Acc.'].append(accuracy(v_y, lr.predict(v_X)))
    
ovr_df = pd.Dataframe(ovr_df)
if not ovr_models_exists:
    joblib.dump(ovr_models, ovr_models_path)